In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols

import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup
import re
import pprint

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_column', 100, 'display.max_row', 1000)

# Exploring the data and miner Cleaning

In [2]:
# df = pd.read_csv("AMD_pro.csv", encoding='cp1252')

In [3]:
df = pd.read_csv('/home/abhishek/Documents/processors/tableExport.csv')
df.shape

(584, 40)

In [4]:
# df.sample(5)

In [5]:
df.columns = [sub.replace(' ', '_') for sub in df.columns]
df.rename(columns = {'Base_Clock' : 'Base_Clock_GHz', 'Max._Boost_Clock_¹_²' : 'Boost_Clock_GHz', 
                     '#_of_CPU_Cores' : 'CPU_Core', '#_of_Threads' : 'Threads',
                     'Processor_Technology_for_CPU_Cores' : 'Chip_lithography','PCI_Express®_Version' : 'PCI_Express_Version',
                     'Max. Operating Temperature (Tjmax)' : 'Maximum_core_temperature', }, inplace = True)

In [6]:
df.Model = [i.replace('™', '') for i in df.Model]
df.Family = [i.replace('™', '') for i in df.Family]

df = df.astype({"Graphics_Model":'str'})
df.Graphics_Model = [i.replace('™', '') for i in df.Graphics_Model]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed:_0                          0 non-null      float64
 1   Unnamed:_1                          584 non-null    object 
 2   Model                               584 non-null    object 
 3   Family                              584 non-null    object 
 4   Line                                584 non-null    object 
 5   Platform                            584 non-null    object 
 6   Product_ID_Tray                     545 non-null    object 
 7   Product_ID_Boxed                    222 non-null    object 
 8   Product_ID_MPK                      59 non-null     object 
 9   Launch_Date                         241 non-null    object 
 10  CPU_Core                            579 non-null    float64
 11  Threads                             524 non-n

# Handeling the Missing Values

In [8]:
df.isnull().sum()

Unnamed:_0                            584
Unnamed:_1                              0
Model                                   0
Family                                  0
Line                                    0
Platform                                0
Product_ID_Tray                        39
Product_ID_Boxed                      362
Product_ID_MPK                        525
Launch_Date                           343
CPU_Core                                5
Threads                                60
Graphics_Core_Count                   329
Base_Clock_GHz                          0
Boost_Clock_GHz                        81
All_Core_Boost_Speed                  570
L1_Cache                              257
L2_Cache                               66
L3_Cache                              207
1kU_Pricing                           542
Unlocked_for_Overclocking              94
Chip_lithography                      102
CPU_Socket                             28
Socket_Count                      

#### Dropped useless features

In [9]:
df.drop(['Unnamed:_0', 'Unnamed:_1', 'Product_ID_Tray', 'Product_ID_Boxed', 'Product_ID_MPK',
         'Line', 'Graphics_Core_Count', 'All_Core_Boost_Speed', 'Socket_Count', '1kU_Pricing',
         'Thermal_Solution_PIB', 'Thermal_Solution_MPK', '*OS_Support', 'Memory_Channels', 'Per_Socket_Mem_BW', 'Graphics_Frequency', 'GPU_Base',
         'Supported_Technologies', 'Workload_Affinity', 'AMD_Configurable_TDP_(cTDP)', 'System_Memory_Specification'], axis=1, inplace = True)

In [10]:
df.shape

(584, 19)

In [11]:
df.isnull().sum()

Model                                   0
Family                                  0
Platform                                0
Launch_Date                           343
CPU_Core                                5
Threads                                60
Base_Clock_GHz                          0
Boost_Clock_GHz                        81
L1_Cache                              257
L2_Cache                               66
L3_Cache                              207
Unlocked_for_Overclocking              94
Chip_lithography                      102
CPU_Socket                             28
PCI_Express_Version                   246
Default_TDP                            19
Max._Operating_Temperature_(Tjmax)    136
System_Memory_Type                     35
Graphics_Model                          0
dtype: int64

## Scrapping the value

### Preparing the data for scraping

In [12]:
model = df.Model
# mc = df.Model

In [13]:


# list_of_things_to_remove = ['AMD ', '\u200b', ' (OEM Only)', ' Processor', ' with Radeon RX Vega 11 Graphics', 
#                             ' Microsoft Surface® Edition',  'with Radeon Vega 8 Graphics', ' Mobile with Radeon Vega 3 Graphics', 
#                             ' with Radeon Vega 11 Graphics', '7th Gen ', '6th Gen ', ' APU', ' with Radeon R7 Graphics and Wraith cooler', 
#                             ' with Radeon R7 Graphics and Near Silent Thermal Solution', 
#                             ' with Radeon HD 8670D', ' with Radeon HD 8650D', ' with Radeon HD 8570D', 
#                             ' with Radeon HD 8550D', ' with Radeon HD 8470D', ' with Radeon HD 8400', 
#                             ' with Radeon HD 8370D', ' with Radeon HD 8330', ' with Radeon HD 8450G', 
#                             ' with Radeon HD 8280', ' with Radeon HD 8240', ' with Radeon HD 8210', 
#                             ' with Radeon R4 Graphics', ' with Radeon R6 Graphics', ' with Radeon R5 Graphics', 
#                             ' with Radeon R3 Graphics', ' with Radeon R7 Graphics', ' with Radeon R7 Graphics', 
#                             ' with Radeon R2 Graphics', ' with Radeon R2 Graphics', ' with Radeon R5 Series', 
#                             ' with Radeon R3 Series',' with Radeon R7 Series', ' with Wraith cooler', 
#                             ' with Near Silent Thermal Solution', 
#                             ]

# # # list_of_things_to_remove = pd.DataFrame(list_of_things_to_remove)
# # df['Model'] = df['Model'].values.tolist()

In [14]:
for i in range (len(model)):
    x=df['Model'][i].partition(" with")
    df['Model'][i] = x[0]

df['Model'] = [i.replace('AMD ', '') for i in df['Model']] 
df['Model'] = [i.replace('\u200b', '') for i in df['Model']]
df['Model'] = [i.replace(' (OEM Only)', '') for i in df['Model']]
df['Model'] = [i.replace(' Processor', '') for i in df['Model']]
df['Model'] = [i.replace(' Microsoft Surface® Edition', '') for i in df['Model']]
df['Model'] = [i.replace(' Mobile with Radeon Vega 3 Graphics', '') for i in df['Model']]
df['Model'] = [i.replace('7th Gen ', '') for i in df['Model']]
df['Model'] = [i.replace('6th Gen ', '') for i in df['Model']]
df['Model'] = [i.replace(' APU', '') for i in df['Model']]

# df['Model'] = [i.replace('TL-56', 'Turion-64-X2-TL-56') for i in df['Model']]
# df['Model'] = [i.replace('TL-64', 'Turion-64-X2-TL-64') for i in df['Model']]
# df['Model'] = [i.replace('TL-60', 'Turion-64-X2-TL-60') for i in df['Model']]
# df['Model'] = [i.replace('TL-52', 'Turion-64-X2-TL-52') for i in df['Model']]

# df['Model'] = [i.replace('3250 HE', 'Opteron 3250 HE') for i in df['Model']]
# df['Model'] = [i.replace('1045T', 'Phenom II X6 1045T') for i in df['Model']]
# df['Model'] = [i.replace('B95', 'Phenom II X4 B95') for i in df['Model']]

df['Model'] = [i.replace(' ', '-') for i in df['Model']]

In [15]:
df.to_csv('pre_finel.csv', index=False)

In [16]:
opteron = df.loc[df['Family'] == 'AMD Opteron']

In [17]:
# opteron

In [18]:
df.head()

,Model,Family,Platform,Launch_Date,CPU_Core,Threads,Base_Clock_GHz,Boost_Clock_GHz,L1_Cache,L2_Cache,L3_Cache,Unlocked_for_Overclocking,Chip_lithography,CPU_Socket,PCI_Express_Version,Default_TDP,Max._Operating_Temperature_(Tjmax),System_Memory_Type,Graphics_Model
0,Ryzen-Threadripper-PRO-5995WX,AMD Ryzen PRO Processors,Desktop,NaN,64.0,128.0,2.7GHz,Up to 4.5GHz,4096KB,32MB,256MB,No,TSMC 7nm FinFET,sWRX8,PCIe 4.0,280W,95°C,DDR4,Discrete Graphics Card Required
1,Ryzen-Threadripper-PRO-5975WX,AMD Ryzen PRO Processors,Desktop,NaN,32.0,64.0,3.6GHz,Up to 4.5GHz,2048KB,16MB,128MB,No,TSMC 7nm FinFET,sWRX8,PCIe 4.0,280W,95°C,DDR4,Discrete Graphics Card Required
2,Ryzen-Threadripper-PRO-5965WX,AMD Ryzen PRO Processors,Desktop,NaN,24.0,48.0,3.8GHz,Up to 4.5GHz,1536KB,12MB,128MB,No,TSMC 7nm FinFET,sWRX8,PCIe 4.0,280W,95°C,DDR4,Discrete Graphics Card Required
3,Ryzen-Threadripper-PRO-5955WX,AMD Ryzen PRO Processors,Desktop,NaN,16.0,32.0,4.0GHz,Up to 4.5GHz,1024KB,8MB,64MB,No,TSMC 7nm FinFET,sWRX8,PCIe 4.0,280W,95°C,DDR4,Discrete Graphics Card Required
4,Ryzen-Threadripper-PRO-5945WX,AMD Ryzen PRO Processors,Desktop,NaN,12.0,24.0,4.1GHz,Up to 4.5GHz,765KB,6MB,64MB,No,TSMC 7nm FinFET,sWRX8,PCIe 4.0,280W,95°C,DDR4,Discrete Graphics Card Required


#### 1st feature that have missing value is 'Launch_Date'

In [19]:
lnch_date = df[df['Launch_Date'].isna()]

In [20]:
releas_date= []
    
for j in lnch_date.Model:
    url = 'https://technical.city/en/cpu/{}'.format(j)
    my_header = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8"}
    r = requests.get(url, headers= my_header).text
    soup = BeautifulSoup(r, 'html.parser')
    
    try:
        parent0 = soup.find_all('div', class_ = 'type-info')
        parent1 = parent0[1].find_all('div', class_ = 'item')
        parent2 = parent1[0].find('strong')
        final = parent2.get_text(strip=True, separator=" ")
        releas_date.append(final)
    
    except:
        releas_date.append(np.NaN)

In [21]:
releas_date = pd.DataFrame(releas_date)

In [22]:
releas_date.isnull().sum()

0    115
dtype: int64

In [23]:
series1=releas_date.iloc[:,0]

In [24]:
series1.shape

(343,)

In [25]:
df.Launch_Date = df['Launch_Date'].fillna(series1)

In [26]:
df.Launch_Date.isnull().sum()

273

In [54]:
df[df['Launch_Date'].isna()].Launch_Date.shape

(273,)

#### 2nd feature that have missing value is '#_of_CPU_Cores'

In [28]:
cpu_nan = df[df['CPU_Core'].isna()]
cpu_nan

,Model,Family,Platform,Launch_Date,CPU_Core,Threads,Base_Clock_GHz,Boost_Clock_GHz,L1_Cache,L2_Cache,L3_Cache,Unlocked_for_Overclocking,Chip_lithography,CPU_Socket,PCI_Express_Version,Default_TDP,Max._Operating_Temperature_(Tjmax),System_Memory_Type,Graphics_Model
414,1045T,AMD Phenom,Boxed Processor,NaN,NaN,NaN,2.7GHz,Up to 3.2GHz,768KB,3MB,6MB,No,NaN,AM3,NaN,95W,71°C,Not Listed,Discrete Graphics Card Required
445,B95,AMD Phenom,Desktop,NaN,NaN,NaN,3.0GHz,NaN,NaN,2MB,NaN,No,NaN,AM3,NaN,95W,NaN,Not Listed,Discrete Graphics Card Required
484,2800+,AMD Sempron,Desktop,NaN,NaN,NaN,56MHz,NaN,NaN,NaN,NaN,No,65nm,939,NaN,45W,70°C,Not Listed,Discrete Graphics Card Required
578,3250-HE,AMD Opteron,Server,NaN,NaN,NaN,2.5GHz,Up to 3.5GHz,192KB,4MB,4MB,No,32nm,AM3+,NaN,45W,NaN,DDR3,nan
582,TL-56,AMD Turion,Laptop,NaN,NaN,NaN,1.8GHz,NaN,NaN,NaN,NaN,No,NaN,S1,NaN,31W,NaN,Not Listed,Discrete Graphics Card Required


In [29]:
# cpu_core = []

# for j in cpu_nan.Model:
#     url = 'https://technical.city/en/cpu/{}'.format(j)
#     my_header = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8"}
#     r = requests.get(url, headers= my_header).text

#     soup = BeautifulSoup(r, 'html.parser')
    
#     try:
#         parent1=soup.select_one('tr:has(> td:-soup-contains("Physical cores"))')
#         parent2 = parent1.find('em', class_ = 'blue')
#         final = parent2.get_text(strip=True, separator=" ")

#         final = re.findall(r'\d+', final)
#         cpu_core.append(final)
        
#     except:
#         cpu_core.append(np.NaN)

In [30]:
df.CPU_Core[414] = 6.0
df.CPU_Core[484] = 1.0
df.CPU_Core[578] = 4.0
df.CPU_Core[445] = 4.0
# df.CPU_Core[578] = 4.0



In [31]:
# df['CPU_Core'] = df['CPU_Core'].fillna([[6.0, 4.0, 8.0, 2.0]])

In [32]:
df['CPU_Core'].isnull().sum()

1

#### 2nd Feature that have NaN! Values is '#_of_Threads'

In [33]:
# df[df['#_of_Threads'].isna()]

In [34]:
# We replaced the NaN by 0 because these processor doesn't have any threads 

# df['#_of_Threads'] = df['#_of_Threads'].fillna(0)

In [35]:
# We converted these feature float to int because counting the cores and threads in float was not making sens

# df = df.astype({"#_of_CPU_Cores":'int', "#_of_Threads":'int'})

#### 3rd Feature that have NaN! Values is "Boost_Clock_GHz"

In [36]:
# df = df.astype({"Boost_Clock_GHz":'str'})

In [37]:
# df.Base_Clock_GHz = [i.replace('GHz', '') for i in df.Base_Clock_GHz]
# df.Boost_Clock_GHz = [i.replace('Up to ', '') for i in df.Boost_Clock_GHz]
# df.Boost_Clock_GHz = [i.replace('GHz', '') for i in df.Boost_Clock_GHz]

In [38]:
# df = df.astype({"Base_Clock_GHz":'float', "Boost_Clock_GHz":'float'})

In [39]:
# base_mean = (df["Boost_Clock_GHz"] - df.Base_Clock_GHz).mean()

In [40]:
# boost_nan = df[df['Boost_Clock_GHz'].isna()]

In [41]:
# boost_fill = base_mean + boost_nan.Base_Clock_GHz


In [42]:
# type(boost_fill)

In [43]:
# df['Boost_Clock_GHz'] = df['Boost_Clock_GHz'].fillna(boost_fill)

In [44]:
# df['Boost_Clock_GHz'].isnull().sum()

#### 4th Feature that have NaN! Values is "'L1_Cache', 'L2_Cache','L3_Cache'" 

In [45]:
# We filled the NaN of these three features with 0 beacause 
# we can count these three coloumn as one feature according 
# to this all the processors are having cache wether it is L1, L2 or L3

# df.update(df[['L1_Cache', 'L2_Cache','L3_Cache']].fillna(0))

#### 5th Feature that have NaN! Values is 'Unlocked_for_Overclocking'

In [46]:
# df[df['Unlocked_for_Overclocking'].isna()].head()

In [47]:
# I checked on google most of processor which is missing this feature 
# are not supporting overclocking that is why we replaced the nan with no
 
# df['Unlocked_for_Overclocking'] = df['Unlocked_for_Overclocking'].fillna('No')

In [48]:
# df['Unlocked_for_Overclocking'].value_counts()

#### 6th Feature that have NaN! Values is 'Processor_Technology_for_CPU_Cores'

In [49]:
# df.Model = [i.replace('™', '') for i in df.Model]
# df.Family = [i.replace('™', '') for i in df.Family]


In [50]:
# amd_epyc = df.loc[df['Family'] == 'AMD EPYC']

In [51]:
# amd_phenom = df.loc[df['Family'] == 'AMD Phenom']

In [52]:
# amd_epyc.Model = [i.replace('AMD EPYC ', '') for i in amd_epyc.Model]

In [53]:
# print(ols(f"price~{i}",data=df).fit().summary())